# Smart Retrieval SLM Analysis

## Hypothesis
**"High-quality retrieval can compensate for smaller, faster LLMs."**

This notebook provides structured analysis of the experiment results, organized by:

### Analysis Sections
1. **Data Loading & Overview** - Load results, parse experiment groups
2. **RAG vs Direct LLM** - Does retrieval help SLMs?
3. **SLM Comparison** - Which small model performs best with RAG?
4. **Retrieval Strategy Analysis** - Component-by-component breakdown:
   - A→B: Effect of Hybrid Retrieval
   - A→C: Effect of Reranking
   - A→D: Effect of Query Transform (HyDE)
   - A→E: Effect of Iterative RAG
   - A→F: Effect of Premium Stack (Hybrid + Rerank)
5. **Embedding Model Comparison** - Does E5-Mistral beat BGE-large?
6. **Cross-Dataset Analysis** - Which strategies generalize?
7. **Statistical Significance** - Confidence in our findings
8. **Recommendations** - Actionable insights

### Experiment Groups
| Group | Strategy | Description |
|-------|----------|-------------|
| Direct | No RAG | Baseline (3 SLMs) |
| A | Simple Dense | BGE-large, top_k=5 |
| B | Hybrid | Dense + BM25 (70/30) |
| C | Reranking | Dense + BGE reranker |
| D | HyDE | Query transformation |
| E | Iterative | Query refinement |
| F | Premium | Hybrid + Rerank |
| G | E5-Mistral | Better embeddings |

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Dict, List, Any, Optional
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Style settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Paths
STUDY_PATH = Path("../outputs/smart_retrieval_slm")

# Metrics to analyze (in order of importance)
METRICS = ['f1', 'exact_match', 'bertscore', 'bleurt']
PRIMARY_METRIC = 'f1'  # Main metric for comparisons

# SLM display names
SLM_NAMES = {
    'llama': 'Llama-3.2-3B',
    'phi3': 'Phi-3-mini',
    'qwen': 'Qwen-2.5-3B',
}

# Group descriptions
GROUP_INFO = {
    'direct': {'name': 'Direct LLM', 'strategy': 'No retrieval'},
    'a': {'name': 'Simple Dense', 'strategy': 'BGE-large embeddings'},
    'b': {'name': 'Hybrid', 'strategy': 'Dense + BM25 (70/30)'},
    'c': {'name': 'Reranking', 'strategy': 'Dense + BGE reranker'},
    'd': {'name': 'HyDE', 'strategy': 'Query transformation'},
    'e': {'name': 'Iterative', 'strategy': 'Query refinement'},
    'f': {'name': 'Premium', 'strategy': 'Hybrid + Rerank'},
    'g': {'name': 'E5-Mistral', 'strategy': 'Better embeddings'},
}

print(f"Study path: {STUDY_PATH}")
print(f"Exists: {STUDY_PATH.exists()}")

## 1. Data Loading & Overview

In [ ]:
def parse_experiment_name(name: str) -> Dict[str, Any]:
    """Parse smart_retrieval_slm experiment names.
    
    Formats:
    - direct_vllm_metallamaLlama3.23BInstruct_concise_none_nq
    - a1_llama_dense_nq
    - b2_phi3_hybrid_triviaqa
    - g1_qwen_e5mistral_dense_hotpotqa
    """
    config = {
        'name': name,
        'group': None,
        'group_num': None,
        'slm': None,
        'slm_display': None,
        'strategy': None,
        'dataset': None,
        'is_direct': False,
        'is_rag': False,
    }
    
    # Handle direct experiments
    if name.startswith('direct_'):
        config['group'] = 'direct'
        config['is_direct'] = True
        config['strategy'] = 'No RAG'
        
        # Extract SLM from direct name
        if 'Llama' in name or 'llama' in name:
            config['slm'] = 'llama'
        elif 'Phi' in name or 'phi' in name:
            config['slm'] = 'phi3'
        elif 'Qwen' in name or 'qwen' in name:
            config['slm'] = 'qwen'
        
        # Extract dataset
        for ds in ['nq', 'triviaqa', 'hotpotqa']:
            if f'_{ds}' in name or name.endswith(ds):
                config['dataset'] = ds
                break
    else:
        # Parse singleton experiment: a1_llama_dense_nq
        config['is_rag'] = True
        parts = name.split('_')
        
        if len(parts) >= 2:
            # First part: group + number (e.g., 'a1', 'b2', 'g1')
            group_part = parts[0]
            if len(group_part) >= 2 and group_part[0].isalpha():
                config['group'] = group_part[0].lower()
                config['group_num'] = group_part[1:] if group_part[1:].isdigit() else None
            
            # Second part: SLM (llama, phi3, qwen)
            slm_part = parts[1].lower()
            if 'llama' in slm_part:
                config['slm'] = 'llama'
            elif 'phi' in slm_part:
                config['slm'] = 'phi3'
            elif 'qwen' in slm_part:
                config['slm'] = 'qwen'
            
            # Strategy from group
            group_strategies = {
                'a': 'Simple Dense',
                'b': 'Hybrid',
                'c': 'Reranking',
                'd': 'HyDE',
                'e': 'Iterative',
                'f': 'Premium',
                'g': 'E5-Mistral',
            }
            config['strategy'] = group_strategies.get(config['group'], 'Unknown')
            
            # Dataset is usually last part
            for ds in ['nq', 'triviaqa', 'hotpotqa']:
                if ds in parts[-1]:
                    config['dataset'] = ds
                    break
    
    # Add display name
    config['slm_display'] = SLM_NAMES.get(config['slm'], config['slm'])
    
    return config


def load_all_results(study_path: Path) -> pd.DataFrame:
    """Load all experiment results into a DataFrame."""
    results = []
    
    if not study_path.exists():
        print(f"Warning: Study path does not exist: {study_path}")
        return pd.DataFrame()
    
    for exp_dir in study_path.iterdir():
        if not exp_dir.is_dir():
            continue
        
        results_file = exp_dir / "results.json"
        if not results_file.exists():
            continue
        
        try:
            with open(results_file) as f:
                data = json.load(f)
            
            # Parse experiment name
            config = parse_experiment_name(data['name'])
            
            # Add metrics
            row = config.copy()
            for metric in METRICS:
                if metric in data:
                    row[metric] = data[metric]
                elif 'metrics' in data and metric in data['metrics']:
                    row[metric] = data['metrics'][metric]
            
            # Add sample count if available
            row['n_samples'] = data.get('n_samples', data.get('num_questions', None))
            
            results.append(row)
        except Exception as e:
            print(f"Error loading {exp_dir.name}: {e}")
    
    df = pd.DataFrame(results)
    
    # Sort by group, slm, dataset
    if not df.empty:
        df = df.sort_values(['group', 'slm', 'dataset']).reset_index(drop=True)
    
    return df


# Load data
df = load_all_results(STUDY_PATH)
print(f"Loaded {len(df)} experiments")

if len(df) > 0:
    print(f"\nGroups: {sorted(df['group'].dropna().unique())}")
    print(f"SLMs: {sorted(df['slm'].dropna().unique())}")
    print(f"Datasets: {sorted(df['dataset'].dropna().unique())}")
    print(f"\nMetrics available: {[m for m in METRICS if m in df.columns]}")

In [ ]:
# Show experiment overview by group
if len(df) > 0:
    overview = df.groupby(['group', 'strategy']).agg({
        'name': 'count',
        'slm': lambda x: ', '.join(sorted(x.dropna().unique())),
        PRIMARY_METRIC: ['mean', 'std'] if PRIMARY_METRIC in df.columns else 'count'
    }).round(3)
    overview.columns = ['n_experiments', 'slms', f'{PRIMARY_METRIC}_mean', f'{PRIMARY_METRIC}_std']
    display(overview)
else:
    print("No results loaded yet. Run experiments first.")

## 2. RAG vs Direct LLM

**Question:** Does retrieval help small language models?

Compare Direct LLM baselines against Group A (Simple Dense RAG).

In [ ]:
def compare_rag_vs_direct(df: pd.DataFrame, metric: str = PRIMARY_METRIC) -> pd.DataFrame:
    """Compare RAG (Group A) vs Direct for each SLM and dataset."""
    if df.empty or metric not in df.columns:
        return pd.DataFrame()
    
    direct = df[df['group'] == 'direct'].copy()
    rag_a = df[df['group'] == 'a'].copy()
    
    comparisons = []
    for slm in df['slm'].dropna().unique():
        for dataset in df['dataset'].dropna().unique():
            direct_score = direct[(direct['slm'] == slm) & (direct['dataset'] == dataset)][metric].values
            rag_score = rag_a[(rag_a['slm'] == slm) & (rag_a['dataset'] == dataset)][metric].values
            
            if len(direct_score) > 0 and len(rag_score) > 0:
                diff = rag_score[0] - direct_score[0]
                pct_change = (diff / direct_score[0]) * 100 if direct_score[0] > 0 else 0
                comparisons.append({
                    'slm': slm,
                    'slm_display': SLM_NAMES.get(slm, slm),
                    'dataset': dataset,
                    f'direct_{metric}': direct_score[0],
                    f'rag_{metric}': rag_score[0],
                    'improvement': diff,
                    'pct_change': pct_change,
                    'rag_helps': diff > 0,
                })
    
    return pd.DataFrame(comparisons)


if len(df) > 0 and PRIMARY_METRIC in df.columns:
    rag_comparison = compare_rag_vs_direct(df)
    if not rag_comparison.empty:
        print("RAG vs Direct LLM Comparison (Group A vs Direct)")
        print("="*60)
        display(rag_comparison.round(3))
        
        # Summary
        helps_pct = rag_comparison['rag_helps'].mean() * 100
        avg_improvement = rag_comparison['improvement'].mean()
        print(f"\n📊 Summary:")
        print(f"   RAG helps in {helps_pct:.0f}% of cases")
        print(f"   Average {PRIMARY_METRIC} improvement: {avg_improvement:+.3f}")
else:
    print("Waiting for results...")

In [ ]:
# Visualize RAG vs Direct
if len(df) > 0 and PRIMARY_METRIC in df.columns:
    rag_comparison = compare_rag_vs_direct(df)
    if not rag_comparison.empty:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot 1: Grouped bar chart by SLM
        ax1 = axes[0]
        slm_summary = rag_comparison.groupby('slm_display').agg({
            f'direct_{PRIMARY_METRIC}': 'mean',
            f'rag_{PRIMARY_METRIC}': 'mean',
        })
        slm_summary.plot(kind='bar', ax=ax1, width=0.7)
        ax1.set_title(f'RAG vs Direct by SLM ({PRIMARY_METRIC})')
        ax1.set_xlabel('Small Language Model')
        ax1.set_ylabel(PRIMARY_METRIC.upper())
        ax1.legend(['Direct', 'RAG'])
        ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
        
        # Plot 2: Improvement by dataset
        ax2 = axes[1]
        pivot = rag_comparison.pivot(index='dataset', columns='slm_display', values='improvement')
        pivot.plot(kind='bar', ax=ax2, width=0.7)
        ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
        ax2.set_title(f'{PRIMARY_METRIC} Improvement (RAG - Direct)')
        ax2.set_xlabel('Dataset')
        ax2.set_ylabel(f'{PRIMARY_METRIC} Improvement')
        ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)
        
        plt.tight_layout()
        plt.show()

## 3. SLM Comparison

**Question:** Which small language model performs best with RAG?

Compare Llama-3.2-3B, Phi-3-mini, and Qwen-2.5-3B across all strategies.

In [ ]:
def compare_slms(df: pd.DataFrame, metric: str = PRIMARY_METRIC) -> pd.DataFrame:
    """Compare SLMs across all RAG strategies."""
    if df.empty or metric not in df.columns:
        return pd.DataFrame()
    
    rag_df = df[df['is_rag'] == True].copy()
    
    # Aggregate by SLM and strategy
    slm_comparison = rag_df.groupby(['slm_display', 'strategy']).agg({
        metric: ['mean', 'std', 'count']
    }).round(3)
    slm_comparison.columns = ['mean', 'std', 'n']
    
    return slm_comparison


if len(df) > 0 and PRIMARY_METRIC in df.columns:
    slm_comp = compare_slms(df)
    if not slm_comp.empty:
        print(f"SLM Performance by Strategy ({PRIMARY_METRIC})")
        print("="*60)
        display(slm_comp)
        
        # Best SLM per strategy
        rag_df = df[df['is_rag'] == True]
        best_per_strategy = rag_df.loc[rag_df.groupby('strategy')[PRIMARY_METRIC].idxmax()][['strategy', 'slm_display', PRIMARY_METRIC]]
        print(f"\n🏆 Best SLM per Strategy:")
        for _, row in best_per_strategy.iterrows():
            print(f"   {row['strategy']}: {row['slm_display']} ({row[PRIMARY_METRIC]:.3f})")

In [ ]:
# Heatmap of SLM x Strategy
if len(df) > 0 and PRIMARY_METRIC in df.columns:
    rag_df = df[df['is_rag'] == True]
    if not rag_df.empty:
        pivot = rag_df.pivot_table(
            index='slm_display', 
            columns='strategy', 
            values=PRIMARY_METRIC, 
            aggfunc='mean'
        )
        
        plt.figure(figsize=(12, 4))
        sns.heatmap(pivot, annot=True, fmt='.3f', cmap='YlGnBu', cbar_kws={'label': PRIMARY_METRIC})
        plt.title(f'SLM x Strategy Performance ({PRIMARY_METRIC})')
        plt.xlabel('Strategy')
        plt.ylabel('Small Language Model')
        plt.tight_layout()
        plt.show()

## 4. Retrieval Strategy Analysis

**Question:** What is the effect of each retrieval component?

We compare each strategy against the baseline (Group A: Simple Dense).

In [ ]:
def component_analysis(df: pd.DataFrame, baseline_group: str = 'a', metric: str = PRIMARY_METRIC) -> pd.DataFrame:
    """Analyze improvement of each strategy over baseline."""
    if df.empty or metric not in df.columns:
        return pd.DataFrame()
    
    baseline = df[df['group'] == baseline_group].copy()
    results = []
    
    for group in df['group'].dropna().unique():
        if group in ['direct', baseline_group]:
            continue
        
        strategy_df = df[df['group'] == group]
        group_name = GROUP_INFO.get(group, {}).get('name', group)
        
        for slm in df['slm'].dropna().unique():
            for dataset in df['dataset'].dropna().unique():
                base_score = baseline[
                    (baseline['slm'] == slm) & (baseline['dataset'] == dataset)
                ][metric].values
                
                strat_score = strategy_df[
                    (strategy_df['slm'] == slm) & (strategy_df['dataset'] == dataset)
                ][metric].values
                
                if len(base_score) > 0 and len(strat_score) > 0:
                    diff = strat_score[0] - base_score[0]
                    pct = (diff / base_score[0]) * 100 if base_score[0] > 0 else 0
                    results.append({
                        'group': group,
                        'strategy': group_name,
                        'slm': slm,
                        'slm_display': SLM_NAMES.get(slm, slm),
                        'dataset': dataset,
                        f'baseline_{metric}': base_score[0],
                        f'strategy_{metric}': strat_score[0],
                        'improvement': diff,
                        'pct_change': pct,
                    })
    
    return pd.DataFrame(results)


if len(df) > 0 and PRIMARY_METRIC in df.columns:
    component_df = component_analysis(df)
    if not component_df.empty:
        # Summary by strategy
        summary = component_df.groupby('strategy').agg({
            'improvement': ['mean', 'std', 'min', 'max'],
            'pct_change': 'mean',
        }).round(3)
        summary.columns = ['mean_improvement', 'std', 'min', 'max', 'avg_pct_change']
        summary = summary.sort_values('mean_improvement', ascending=False)
        
        print(f"Strategy Improvements over Simple Dense (Group A)")
        print("="*60)
        display(summary)
        
        # Ranking
        print(f"\n🏆 Strategy Ranking (by avg {PRIMARY_METRIC} improvement):")
        for i, (strategy, row) in enumerate(summary.iterrows(), 1):
            emoji = "✅" if row['mean_improvement'] > 0 else "❌"
            print(f"   {i}. {strategy}: {row['mean_improvement']:+.3f} ({row['avg_pct_change']:+.1f}%) {emoji}")

In [ ]:
# Visualize component effects
if len(df) > 0 and PRIMARY_METRIC in df.columns:
    component_df = component_analysis(df)
    if not component_df.empty:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot 1: Average improvement by strategy
        ax1 = axes[0]
        strategy_means = component_df.groupby('strategy')['improvement'].mean().sort_values(ascending=True)
        colors = ['green' if x > 0 else 'red' for x in strategy_means.values]
        strategy_means.plot(kind='barh', ax=ax1, color=colors)
        ax1.axvline(x=0, color='black', linestyle='--', alpha=0.5)
        ax1.set_title(f'Average {PRIMARY_METRIC} Improvement over Simple Dense')
        ax1.set_xlabel(f'{PRIMARY_METRIC} Improvement')
        ax1.set_ylabel('Strategy')
        
        # Plot 2: Box plot of improvements by strategy
        ax2 = axes[1]
        order = component_df.groupby('strategy')['improvement'].mean().sort_values(ascending=False).index
        sns.boxplot(data=component_df, x='strategy', y='improvement', order=order, ax=ax2)
        ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
        ax2.set_title(f'{PRIMARY_METRIC} Improvement Distribution')
        ax2.set_xlabel('Strategy')
        ax2.set_ylabel(f'{PRIMARY_METRIC} Improvement')
        ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
        
        plt.tight_layout()
        plt.show()

### 4.1 Detailed Component Breakdown

Let's examine each component's effect in detail.

In [ ]:
def print_component_detail(component_df: pd.DataFrame, strategy: str, description: str):
    """Print detailed analysis for a specific strategy."""
    subset = component_df[component_df['strategy'] == strategy]
    if subset.empty:
        print(f"No data for {strategy}")
        return
    
    print(f"\n{'='*60}")
    print(f"📊 {strategy}: {description}")
    print(f"{'='*60}")
    
    # By SLM
    by_slm = subset.groupby('slm_display')['improvement'].agg(['mean', 'std'])
    print(f"\nBy SLM:")
    for slm, row in by_slm.iterrows():
        emoji = "✅" if row['mean'] > 0 else "❌"
        print(f"   {slm}: {row['mean']:+.3f} ± {row['std']:.3f} {emoji}")
    
    # By Dataset
    by_ds = subset.groupby('dataset')['improvement'].agg(['mean', 'std'])
    print(f"\nBy Dataset:")
    for ds, row in by_ds.iterrows():
        emoji = "✅" if row['mean'] > 0 else "❌"
        print(f"   {ds}: {row['mean']:+.3f} ± {row['std']:.3f} {emoji}")
    
    # Overall
    overall = subset['improvement'].mean()
    overall_pct = subset['pct_change'].mean()
    print(f"\nOverall: {overall:+.3f} ({overall_pct:+.1f}%)")


if len(df) > 0 and PRIMARY_METRIC in df.columns:
    component_df = component_analysis(df)
    if not component_df.empty:
        print_component_detail(component_df, 'Hybrid', 'Dense + BM25 (70/30) keyword matching')
        print_component_detail(component_df, 'Reranking', 'Cross-encoder reranking (fetch 25 → keep 5)')
        print_component_detail(component_df, 'HyDE', 'Hypothetical Document Embeddings')
        print_component_detail(component_df, 'Iterative', 'Query refinement with 1 iteration')
        print_component_detail(component_df, 'Premium', 'Hybrid + Reranking combined')

## 5. Embedding Model Comparison

**Question:** Does E5-Mistral-7B (top MTEB) outperform BGE-large?

Compare Group G (E5-Mistral) against equivalent experiments with BGE-large.

In [ ]:
def compare_embeddings(df: pd.DataFrame, metric: str = PRIMARY_METRIC) -> pd.DataFrame:
    """Compare E5-Mistral (Group G) vs BGE-large (Group A)."""
    if df.empty or metric not in df.columns:
        return pd.DataFrame()
    
    # G1 vs A3 (Qwen), G3 vs A2 (Phi-3), G4 vs A1 (Llama)
    bge = df[df['group'] == 'a'].copy()
    e5 = df[df['group'] == 'g'].copy()
    
    # Filter G to only dense experiments (not g2 which has reranking)
    e5_dense = e5[~e5['name'].str.contains('rerank', case=False, na=False)]
    
    comparisons = []
    for slm in df['slm'].dropna().unique():
        for dataset in df['dataset'].dropna().unique():
            bge_score = bge[(bge['slm'] == slm) & (bge['dataset'] == dataset)][metric].values
            e5_score = e5_dense[(e5_dense['slm'] == slm) & (e5_dense['dataset'] == dataset)][metric].values
            
            if len(bge_score) > 0 and len(e5_score) > 0:
                diff = e5_score[0] - bge_score[0]
                comparisons.append({
                    'slm': slm,
                    'slm_display': SLM_NAMES.get(slm, slm),
                    'dataset': dataset,
                    f'bge_large_{metric}': bge_score[0],
                    f'e5_mistral_{metric}': e5_score[0],
                    'improvement': diff,
                    'e5_better': diff > 0,
                })
    
    return pd.DataFrame(comparisons)


if len(df) > 0 and PRIMARY_METRIC in df.columns:
    embed_comp = compare_embeddings(df)
    if not embed_comp.empty:
        print("E5-Mistral-7B vs BGE-large Comparison")
        print("="*60)
        display(embed_comp.round(3))
        
        # Summary
        e5_wins = embed_comp['e5_better'].mean() * 100
        avg_diff = embed_comp['improvement'].mean()
        print(f"\n📊 Summary:")
        print(f"   E5-Mistral wins in {e5_wins:.0f}% of cases")
        print(f"   Average improvement: {avg_diff:+.3f}")
        
        if avg_diff > 0:
            print(f"   ✅ E5-Mistral-7B is better overall")
        else:
            print(f"   ❌ BGE-large is sufficient (E5 not worth the cost)")
    else:
        print("No E5-Mistral experiments found in Group G yet.")

## 6. Cross-Dataset Analysis

**Question:** Which strategies generalize across datasets?

Some strategies may work well on NQ but fail on HotpotQA (multi-hop).

In [ ]:
if len(df) > 0 and PRIMARY_METRIC in df.columns:
    # Pivot: Strategy x Dataset
    rag_df = df[df['is_rag'] == True]
    if not rag_df.empty:
        pivot = rag_df.pivot_table(
            index='strategy',
            columns='dataset',
            values=PRIMARY_METRIC,
            aggfunc='mean'
        )
        
        # Add variance across datasets (lower = more consistent)
        pivot['variance'] = pivot.var(axis=1)
        pivot['consistent'] = pivot['variance'] < pivot['variance'].median()
        pivot = pivot.sort_values('variance')
        
        print(f"Strategy Performance by Dataset ({PRIMARY_METRIC})")
        print("="*60)
        display(pivot.round(3))
        
        # Most consistent strategies
        print(f"\n🎯 Most Consistent Strategies (lowest variance):")
        for strategy in pivot.head(3).index:
            var = pivot.loc[strategy, 'variance']
            print(f"   {strategy}: variance={var:.4f}")

In [ ]:
# Heatmap of Strategy x Dataset
if len(df) > 0 and PRIMARY_METRIC in df.columns:
    rag_df = df[df['is_rag'] == True]
    if not rag_df.empty:
        pivot = rag_df.pivot_table(
            index='strategy',
            columns='dataset',
            values=PRIMARY_METRIC,
            aggfunc='mean'
        )
        
        plt.figure(figsize=(10, 6))
        sns.heatmap(pivot, annot=True, fmt='.3f', cmap='RdYlGn', center=pivot.values.mean())
        plt.title(f'Strategy Performance by Dataset ({PRIMARY_METRIC})')
        plt.xlabel('Dataset')
        plt.ylabel('Strategy')
        plt.tight_layout()
        plt.show()

## 7. Statistical Significance

**Question:** Are our findings statistically significant?

We use paired t-tests to compare strategies.

In [ ]:
def statistical_tests(df: pd.DataFrame, metric: str = PRIMARY_METRIC) -> pd.DataFrame:
    """Perform statistical tests comparing strategies."""
    if df.empty or metric not in df.columns:
        return pd.DataFrame()
    
    baseline = df[df['group'] == 'a']
    results = []
    
    for group in ['b', 'c', 'd', 'e', 'f', 'g']:
        strategy_df = df[df['group'] == group]
        if strategy_df.empty:
            continue
        
        strategy_name = GROUP_INFO.get(group, {}).get('name', group)
        
        # Match by slm and dataset
        paired_scores = []
        for slm in df['slm'].dropna().unique():
            for dataset in df['dataset'].dropna().unique():
                base = baseline[(baseline['slm'] == slm) & (baseline['dataset'] == dataset)][metric].values
                strat = strategy_df[(strategy_df['slm'] == slm) & (strategy_df['dataset'] == dataset)][metric].values
                if len(base) > 0 and len(strat) > 0:
                    paired_scores.append((base[0], strat[0]))
        
        if len(paired_scores) >= 3:  # Need at least 3 pairs
            base_scores = [p[0] for p in paired_scores]
            strat_scores = [p[1] for p in paired_scores]
            
            t_stat, p_value = stats.ttest_rel(strat_scores, base_scores)
            mean_diff = np.mean(strat_scores) - np.mean(base_scores)
            
            results.append({
                'strategy': strategy_name,
                'n_pairs': len(paired_scores),
                'mean_difference': mean_diff,
                't_statistic': t_stat,
                'p_value': p_value,
                'significant_05': p_value < 0.05,
                'significant_01': p_value < 0.01,
            })
    
    return pd.DataFrame(results)


if len(df) > 0 and PRIMARY_METRIC in df.columns:
    stat_results = statistical_tests(df)
    if not stat_results.empty:
        print(f"Statistical Significance (paired t-test vs Simple Dense)")
        print("="*60)
        display(stat_results.round(4))
        
        sig_05 = stat_results[stat_results['significant_05']]['strategy'].tolist()
        if sig_05:
            print(f"\n✅ Significant at p<0.05: {', '.join(sig_05)}")
        else:
            print(f"\n❌ No strategies significantly better at p<0.05")

## 8. Recommendations

Based on the analysis, here are the actionable insights.

In [ ]:
def generate_recommendations(df: pd.DataFrame, metric: str = PRIMARY_METRIC):
    """Generate recommendations based on analysis."""
    if df.empty or metric not in df.columns:
        print("No data available for recommendations.")
        return
    
    print("="*60)
    print("📋 RECOMMENDATIONS")
    print("="*60)
    
    rag_df = df[df['is_rag'] == True]
    direct_df = df[df['is_direct'] == True]
    
    # 1. Best overall configuration
    if not rag_df.empty:
        best_idx = rag_df[metric].idxmax()
        best = rag_df.loc[best_idx]
        print(f"\n1. BEST OVERALL CONFIGURATION")
        print(f"   Strategy: {best['strategy']}")
        print(f"   SLM: {best['slm_display']}")
        print(f"   {metric}: {best[metric]:.3f}")
    
    # 2. Best SLM
    if not rag_df.empty:
        slm_avg = rag_df.groupby('slm_display')[metric].mean().sort_values(ascending=False)
        print(f"\n2. BEST SLM (averaged across strategies)")
        for i, (slm, score) in enumerate(slm_avg.items(), 1):
            emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
            print(f"   {emoji} {slm}: {score:.3f}")
    
    # 3. Most impactful component
    component_df = component_analysis(df)
    if not component_df.empty:
        strategy_impact = component_df.groupby('strategy')['improvement'].mean().sort_values(ascending=False)
        print(f"\n3. MOST IMPACTFUL COMPONENTS (vs Simple Dense)")
        for strategy, impact in strategy_impact.head(3).items():
            emoji = "✅" if impact > 0 else "❌"
            print(f"   {emoji} {strategy}: {impact:+.3f}")
    
    # 4. RAG vs Direct summary
    if not rag_df.empty and not direct_df.empty:
        rag_avg = rag_df[metric].mean()
        direct_avg = direct_df[metric].mean()
        print(f"\n4. RAG VALUE")
        print(f"   Direct LLM avg: {direct_avg:.3f}")
        print(f"   RAG avg: {rag_avg:.3f}")
        print(f"   Improvement: {rag_avg - direct_avg:+.3f} ({((rag_avg/direct_avg)-1)*100:+.1f}%)")
    
    # 5. Cost-effectiveness
    print(f"\n5. COST-EFFECTIVENESS RANKING")
    print(f"   (Best performance-to-complexity ratio)")
    complexity = {
        'Simple Dense': 1,
        'Hybrid': 2,
        'Reranking': 3,
        'HyDE': 4,
        'Iterative': 5,
        'Premium': 6,
        'E5-Mistral': 7,
    }
    if not component_df.empty:
        strategy_scores = component_df.groupby('strategy')['improvement'].mean()
        cost_eff = [(s, strategy_scores.get(s, 0) / complexity.get(s, 1)) 
                    for s in complexity.keys() if s in strategy_scores.index]
        cost_eff.sort(key=lambda x: x[1], reverse=True)
        for strategy, eff in cost_eff[:3]:
            print(f"   - {strategy}: efficiency={eff:.3f}")


if len(df) > 0 and PRIMARY_METRIC in df.columns:
    generate_recommendations(df)
else:
    print("Run experiments first to generate recommendations.")

## 9. All Metrics Comparison

Let's see the full picture with all metrics.

In [ ]:
if len(df) > 0:
    # Summary table with all metrics
    available_metrics = [m for m in METRICS if m in df.columns]
    
    if available_metrics:
        rag_df = df[df['is_rag'] == True]
        if not rag_df.empty:
            summary = rag_df.groupby('strategy')[available_metrics].mean().round(3)
            summary = summary.sort_values(PRIMARY_METRIC, ascending=False)
            
            print("All Metrics Summary by Strategy")
            print("="*60)
            display(summary)
            
            # Correlation between metrics
            if len(available_metrics) > 1:
                print(f"\nMetric Correlations:")
                corr = rag_df[available_metrics].corr()
                display(corr.round(3))

## 10. Export Results

In [ ]:
# Export processed results
if len(df) > 0:
    output_dir = STUDY_PATH / "analysis"
    output_dir.mkdir(exist_ok=True)
    
    # Full results
    df.to_csv(output_dir / "full_results.csv", index=False)
    
    # Component analysis
    if PRIMARY_METRIC in df.columns:
        component_df = component_analysis(df)
        if not component_df.empty:
            component_df.to_csv(output_dir / "component_analysis.csv", index=False)
    
    print(f"Results exported to: {output_dir}")